In [3]:
# Travel time cell
from datetime import datetime, timedelta
from pytz import timezone as ptz

DATE_FORMAT = "%Y-%m-%dT%H:%M:%-SZ"
oneDay = timedelta(1)

departureTimeOptions = {
    'now': lambda tz: datetime.now(tz).replace(minute=59, second=59).isoformat(),
    'morning': lambda tz: (datetime.now(tz).replace(hour=9, minute=0, second=0) + oneDay).isoformat(),
    'noon': lambda tz: (datetime.now(tz).replace(hour=12, minute=0, second=0) + oneDay).isoformat(),
    'afternoon': lambda tz: (datetime.now(tz).replace(hour=5, minute=0, second=0) + oneDay).isoformat(),
    'evening': lambda tz: (datetime.now(tz).replace(hour=7, minute=0, second=0) + oneDay).isoformat(),
}

print("now time: " + departureTimeOptions['now'](ptz('EST')))
print("morning time: " + departureTimeOptions['morning'](ptz('EST')))
print(datetime.now().timestamp())

now time: 2023-10-05T20:59:59.731428-05:00
morning time: 2023-10-06T09:00:00.733173-05:00
1696556153.733288


In [4]:
import math
import requests
from const import GOOGLE_MAPS_API_KEY
import json


COMPUTE_ROUTE_URL = 'https://routes.googleapis.com/directions/v2:computeRoutes'
MY_HOUSE = '3025 The Credit Woodlands, Mississauga, ON L5C 2V3'
SQUARE_ONE_ADDRESS = '100 City Centre Dr, Mississauga, ON L5B 2C9'
UNIVERSITY_OF_TORONTO = "27 King's College Cir, Toronto, ON M5S 1A1"
TRAVEL_MODES = ['DRIVE', 'TRANSIT', 'BICYCLE']


def getDurationSec(route): return int(route["duration"][0:-1])


def findFastestRoute(response) -> timedelta:
    routeDurations = json.loads(response.text)["routes"]
    fastestCommute = math.inf

    for r in routeDurations:
        fastestCommute = min(getDurationSec(r), fastestCommute)

    return timedelta(seconds=fastestCommute)


DISTANCE_AND_LINE_FIELDMASK_HEADER = {
    'Content-Type': 'application/json',
    'X-Goog-Api-Key': GOOGLE_MAPS_API_KEY,
    'X-Goog-FieldMask': 'routes.duration,routes.distanceMeters,routes.polyline.encodedPolyline',
}

DURATION_FIELDMASK_HEADER = {
    'Content-Type': 'application/json',
    'X-Goog-Api-Key': GOOGLE_MAPS_API_KEY,
    'X-Goog-FieldMask': 'routes.duration',
}

TRANSIT_DETAIL_FIELDMASK_HEADER = {
    'content-type': 'application/json; application/json',
    'X-Goog-Api-Key': GOOGLE_MAPS_API_KEY,
    'X-Goog-FieldMask': 'routes.legs.steps.transitDetails',
}


def driveRouteParams(originAddress: str, destinationAddress: str, departureOption='now') -> dict:
    return {
        'origin': {
            'address': originAddress
        },
        'destination': {
            'address': destinationAddress
        },
        'travelMode': 'DRIVE',
        'routingPreference': 'TRAFFIC_AWARE',
        'departureTime': departureTimeOptions[departureOption](ptz('EST')),
        'computeAlternativeRoutes': False,
        'routeModifiers': {
            'avoidTolls': True,
            'avoidHighways': False,
            'avoidFerries': True,
        },
    }


def transitRouteParam(originAddress: str, destinationAddress: str, departureOption='now'):
    return {
        'origin': {
            'address': originAddress
        },
        'destination': {
            'address': destinationAddress
        },
        'travelMode': 'TRANSIT',
        'computeAlternativeRoutes': True,
        'transitPreferences': {
            'routingPreference': 'FEWER_TRANSFERS',
        },
        'departureTime': departureTimeOptions[departureOption](ptz('EST')),
    }


def bicycleRouteParam(originAddress: str, destinationAddress: str, departureOption='now'):
    return {
        'origin': {
            'address': originAddress
        },
        'destination': {
            'address': destinationAddress
        },
        'travelMode': 'BICYCLE',
        'departureTime': departureTimeOptions[departureOption](ptz('EST')),
    }


def computeRoute(fieldMask, travelMode, orginAddr, destAddr, departureOption='now'):
    travelParam = {
        'DRIVE': driveRouteParams,
        'TRANSIT': transitRouteParam,
        'BICYCLE': bicycleRouteParam,
    }
    return requests.post(COMPUTE_ROUTE_URL, headers=fieldMask, json=travelParam[travelMode](orginAddr, destAddr, departureOption))


def commuteTime(travelMode, orginAddr, destAddr, departureOption='now'):
    return findFastestRoute(computeRoute(DURATION_FIELDMASK_HEADER, travelMode, orginAddr, destAddr, departureOption))


print(computeRoute(DISTANCE_AND_LINE_FIELDMASK_HEADER,
      'DRIVE', MY_HOUSE, SQUARE_ONE_ADDRESS).text)

print(
    f'travel times from My_House: ${MY_HOUSE} to Square One ${SQUARE_ONE_ADDRESS}')
print("Fastest Driving Commute: ")
print(commuteTime('DRIVE', MY_HOUSE, SQUARE_ONE_ADDRESS))

print("Fastest transit commute: ")
print(commuteTime('TRANSIT', MY_HOUSE, SQUARE_ONE_ADDRESS))


print("Fastest Bicycle Commute:")
print(str(commuteTime('BICYCLE', MY_HOUSE, SQUARE_ONE_ADDRESS)))

{
  "routes": [
    {
      "distanceMeters": 6812,
      "duration": "844s",
      "polyline": {
        "encodedPolyline": "omyhGtlceNkCuD?Gf@{@LCXN|EtFXd@DRCXq@tAb@l@tA{EF]u@g@uAw@cB}AqBaCSIK?qCeDsCcDiDcE{@}@gFoGoBmC}GcIkEkFqCaDkCkDqJiL}@eAc@cAiJuKeDzEg@~@Of@qEnHyG|K_@Tw@fAkEbH}GzK]b@Kd@m@fA_KbPcC~DoCjEwF`JmAtAsAvAUHc@f@oDdFyCuDeBoBaGqGiLaNiFoGyCkDoRsU{LqOgIpOGRURGDgAD"
      }
    }
  ]
}

travel times from My_House: $3025 The Credit Woodlands, Mississauga, ON L5C 2V3 to Square One $100 City Centre Dr, Mississauga, ON L5B 2C9
Fastest Driving Commute: 
0:14:04
Fastest transit commute: 
0:38:55
Fastest Bicycle Commute:
0:27:16


In [80]:
# Write User Data to a to user_name.csv
# - write important addresses
import pandas as pd

importantLocations = {
    'Family House': MY_HOUSE,
    'Square One': SQUARE_ONE_ADDRESS,
    'UofT': UNIVERSITY_OF_TORONTO
}

data = {
    'ShortName': ['Family House', 'Square One', 'UofT'],
    'Address': [MY_HOUSE, SQUARE_ONE_ADDRESS, UNIVERSITY_OF_TORONTO]
}


userDf = pd.DataFrame({
    'ShortName': ['Family House', 'Square One', 'UofT'],
    'Address': [MY_HOUSE, SQUARE_ONE_ADDRESS, UNIVERSITY_OF_TORONTO]
})
print(userDf)
# userDf.set_index('ShortName')


userDf.to_csv('user_ScroogeMcDuck.csv')

      ShortName                                            Address
0  Family House  3025 The Credit Woodlands, Mississauga, ON L5C...
1    Square One        100 City Centre Dr, Mississauga, ON L5B 2C9
2          UofT         27 King's College Cir, Toronto, ON M5S 1A1


In [5]:
import pandas as pd


def getListOfAddresses(df):
    newDf = pd.DataFrame({})

    newDf['Listing Address'] = df['Address'] + ', ' + \
        df['City'] + ', ' + df['Province'] + ' ' + df['Postal Code']
    return newDf


def computeCommuteMatrix(userAddressesDf, listingsDf):
    commuteMatrixDf = pd.DataFrame(listingsDf)
    for userAddr in userAddressesDf.values:
        for travelMode in TRAVEL_MODES:
            commuteName = travelMode + ' to ' + userAddr[1]
            listingsAddrs = listingsDf.loc[:, 'Listing Address']

            commuteMatrixDf[commuteName] = listingsAddrs.map(
                lambda addr: str(commuteTime(travelMode, addr, userAddr[2])))
            
    return commuteMatrixDf


housingDf = pd.read_csv("Kitchener Waterloo Housing.csv")

houseListings = getListOfAddresses(housingDf)

userDf = pd.read_csv('user_ScroogeMcDuck.csv')

commuteMatrix = computeCommuteMatrix(userDf, houseListings)

commuteMatrix.to_csv('houseListingCommuteTimes.csv')


print(" ")
print("Commute matrix")
print(commuteMatrix)

 
Commute matrix
                               Listing Address DRIVE to Family House  \
0      98 Foxglove Cres, Kitchener, ON N2E 3Y8               1:00:43   
1    228 Bankside Drive, Kitchener, ON N2N 3E5               1:01:36   
2  240 Westmeadow Drive, Kitchener, ON N2N 0A1               1:03:45   

  TRANSIT to Family House BICYCLE to Family House DRIVE to Square One  \
0                 3:21:55                 4:57:55             0:56:50   
1                 3:15:04                 5:02:55             0:57:26   
2                 2:40:26                 5:07:25             0:59:56   

  TRANSIT to Square One BICYCLE to Square One DRIVE to UofT TRANSIT to UofT  \
0               2:51:52               5:05:17       1:17:07         6:46:31   
1               6:18:54               5:10:17       1:18:04         6:33:47   
2               2:50:31               5:14:47       1:19:36         6:44:03   

  BICYCLE to UofT  
0         6:32:21  
1         6:37:21  
2         6:41:52  
